In [6]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 42

In [2]:
import nbimporter

import pre_processing
import feature_generation
import feature_selection
import parameter_tuning
import predict

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb
Importing Jupyter notebook from parameter_tuning.ipynb
Importing Jupyter notebook from predict.ipynb


# Parameter Tuning para XGBoost


In [3]:
import xgboost
from sklearn.model_selection import GridSearchCV

In [4]:
train,test = pre_processing.load_featured_datasets()

In [7]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [8]:
train_selected = feature_selection.get_selected_dataframe(train)
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [9]:
X = train_selected.drop('precio', axis=1).values
Y = train_selected['precio'].values

In [11]:
reg = xgboost.XGBRegressor()

In [12]:
param_grid = {
    'max_depth':[13,14,15],
    'n_estimators':[120,130,140],
    'learning_rate': [0.05,0.1,0.3],
    'subsample':[0.5,0.8,0.9],
    'min_child_weight':[15,20]
}

In [19]:
gridsearch = GridSearchCV(reg, param_grid, cv=4, scoring = 'neg_mean_absolute_error')

In [ ]:
gridsearch.fit(X,Y)